## Tarea 2
- Usar anomalies python package
- Implementar multiprocessing para calcular las soluciones

In [38]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from anomalies import anomaly
from lk import generate_lk

import pandas as pd
import multiprocessing as multiprocessing
import dask.array as da


In [39]:

def compute_z(lk, zmax=30):
    
    m = int(len(lk)/2)
    
    l, k = lk[:m].flatten(), lk[m:].flatten()
    
    z = anomaly.free(l, k)
    
    if np.any(np.isin(z, 0)) or np.any( np.abs(z) >= zmax+1) or np.any(np.isin(z, -z)):

        return {}

    else:

        gcd = anomaly.free.gcd
    
        if gcd==0:
            z_simplified = z
        else:
            z_simplified = anomaly.free.simplified

        if z[0]<0:
            z=(-1)*z
            z_simplified=(-1)*z_simplified
        
        results = {
            'l': l.tolist(),
            'k': k.tolist(),
            'z': z.tolist(),
            'gcd': gcd,
            'z_simplified': z_simplified.tolist()
        }   
    
        return results


In [40]:
def find_all_z(n, lkmax, N):

    lk = generate_lk(n, lkmax, N)
    lk = pd.DataFrame(lk.compute()).drop_duplicates(ignore_index=True).to_numpy()

    pool = multiprocessing.Pool()
    results = pool.map(compute_z, lk)

    return results


In [41]:
def solution_set(n, lkmax, N):
    
    zs = find_all_z(n, lkmax, N)

    all_zs = [d for d in zs if d]

    df = pd.DataFrame(all_zs)
    df = df.sort_values(by = ['gcd'], ignore_index=True)

    df['copy'] = df['z_simplified'].astype(str)
    df = df.drop_duplicates('copy').drop('copy', axis='columns').reset_index(drop=True)

    return df    

In [42]:
df_5 = solution_set(5, 30, 1000000)
df_5

,l,k,z,gcd,z_simplified
0,[1],"[-1, -5]","[2, 18, -23, -25, 28]",1,"[2, 18, -23, -25, 28]"
1,[-4],"[1, -2]","[4, 9, -14, -25, 26]",1,"[4, 9, -14, -25, 26]"
2,[2],"[-1, -2]","[2, 4, -7, -9, 10]",1,"[2, 4, -7, -9, 10]"
3,[-1],"[2, 1]","[2, 10, -14, -16, 18]",2,"[1, 5, -7, -8, 9]"


In [44]:
df_6 = solution_set(6, 11, 1000000)
df_6

,l,k,z,gcd,z_simplified
0,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,"[1, -2, -3, 5, 5, -6]"
1,"[1, -1]","[-2, -3]","[1, -2, -10, 12, 15, -16]",1,"[1, -2, -10, 12, 15, -16]"
2,"[-1, -2]","[-2, 3]","[2, 3, 6, -13, -17, 19]",1,"[2, 3, 6, -13, -17, 19]"
3,"[-1, 2]","[3, -2]","[7, -9, -12, 15, 20, -21]",1,"[7, -9, -12, 15, 20, -21]"
4,"[-1, 2]","[3, 2]","[3, -5, -8, 11, 15, -16]",1,"[3, -5, -8, 11, 15, -16]"
5,"[2, 3]","[1, -3]","[7, -9, -11, 14, 18, -19]",1,"[7, -9, -11, 14, 18, -19]"
6,"[-2, -1]","[-7, -9]","[1, 5, 6, -14, -19, 21]",1,"[1, 5, 6, -14, -19, 21]"
7,"[-2, -1]","[4, 5]","[2, -4, -5, 8, 10, -11]",1,"[2, -4, -5, 8, 10, -11]"
8,"[1, 3]","[2, -2]","[1, -5, -10, 17, 19, -22]",1,"[1, -5, -10, 17, 19, -22]"
9,"[2, -1]","[-4, -3]","[2, 3, 10, -19, -20, 24]",1,"[2, 3, 10, -19, -20, 24]"
